In [1]:
import processor
from data import Data, preprocess_midi_files_under
import torch
from model import load_model
import random
from midi_data import get_tokens
import midi_data as md
import json

In [20]:
preprocess_midi_files_under(f'C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers', 'preprocess_7_composers')

 [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, 15 Sinfonias, BWV 787-801, ntLGHRX5XOE.mid]

 [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, Air in F major, BWV Anh.131, ofljU7lmZMA.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, Allemande in C minor, BWV 834, 0lx-kPXgPw8.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, Allemande in G minor, BWV 836, mCCux8jvhkA.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, Als der gu╠êtige Gott vollenden wollt sein Wort, BWV 264, I69tdLLbIyw.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, Andante in G minor, BWV 969, IN5Ma07uT5M.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_7_composers\Bach, Johann Sebastian, Applicatio in C major, BWV 994, eT0Qcf4C0fI.mid] [C:/Users/Draco/Documents/GitHub/MusicTransfo

In [5]:
# Example usage:
config = processor.load_config(filepath="config/config_thor.yaml")
batch_size = config["batch_size"]
block_size = config["block_size"]
max_iters = config["max_iters"]
eval_interval = config["eval_interval"]
save_interval = config["save_interval"]
learning_rate = config["learning_rate"]
eval_iters = config["eval_iters"]
n_embd = config["n_embd"]
n_head = config["n_head"]
n_layer = config["n_layer"]
dropout = config["dropout"]
vocab_size = config["vocab_size"]
device = config["device"]
generated_event_length = config["generated_event_length"]

In [3]:
train_data, val_data = Data('C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/preprocess_7_composers').train_test_data()
# data = torch.tensor(get_tokens())

In [ ]:
torch.manual_seed(1337)

m = load_model()

def shift_sequence(sequence, rand_ints, lower_bound, upper_bound):
    shifted_sequence = sequence.clone()
    for i in range(sequence.size(0)):
        mask = (sequence[i] >= lower_bound) & (sequence[i] < upper_bound)
        shifted_sequence[i, mask] = torch.clamp(sequence[i, mask] + rand_ints[i], min=lower_bound, max=upper_bound - 1)
    return shifted_sequence

def multiply_sequence(sequence, rand_ints, lower_bound, upper_bound):
    multiplied_sequence = sequence.clone()
    for i in range(sequence.size(0)):
        mask = (sequence[i] >= lower_bound) & (sequence[i] < upper_bound)
        multiplied_sequence[i, mask] = torch.clamp((sequence[i, mask] - lower_bound) * rand_ints[i] + lower_bound, min=lower_bound, max=upper_bound - 1)
    return multiplied_sequence

# Data loading with metadata integration
def get_batch(split):
    # Select train or validation data
    data = train_data if split == 'train' else val_data
    batch_sequences = random.sample(data, batch_size)
    
    # Separate sequences and metadata
    sequences = [item[0] for item in batch_sequences]
    metadata = {"composer": torch.tensor([item[1]["composer"] for item in batch_sequences])}

    # Generate random starting indices for each sequence in the batch
    ix = [random.randint(0, len(seq) - block_size - 1) for seq in sequences]

    # Process sequences for input (x) and target (y)
    new_x = torch.stack([torch.tensor(seq[ix[i]:ix[i] + block_size]) for i, seq in enumerate(sequences)])
    new_y = torch.stack([torch.tensor(seq[ix[i] + 1:ix[i] + block_size + 1]) for i, seq in enumerate(sequences)])

    # Pitch shifting
    note_r_ints = [random.randint(-12, 12) for _ in range(batch_size)]
    note_lb = processor.START_IDX["notes"]
    note_ub = processor.START_IDX["notes"] + processor.RANGE_NOTES
    new_x = shift_sequence(new_x, note_r_ints, note_lb, note_ub)
    new_y = shift_sequence(new_y, note_r_ints, note_lb, note_ub)
    
    # Velocity shifting
    vel_r_ints = [random.randint(-20, 20) for _ in range(batch_size)]
    vel_lb = processor.START_IDX["velocity"]
    vel_ub = processor.START_IDX["velocity"] + processor.RANGE_VEL
    new_x = shift_sequence(new_x, vel_r_ints, vel_lb, vel_ub)
    new_y = shift_sequence(new_y, vel_r_ints, vel_lb, vel_ub)
    
    # Time multiplication
    time_r_ints = [random.randint(1, 3) for _ in range(batch_size)]
    time_lb = processor.START_IDX["time_shift"]
    time_ub = processor.START_IDX["time_shift"] + processor.RANGE_TIME_SHIFT
    new_x = multiply_sequence(new_x, time_r_ints, time_lb, time_ub)
    new_y = multiply_sequence(new_y, time_r_ints, time_lb, time_ub)
    
    # Length multiplication
    len_lb = processor.START_IDX["length"]
    len_ub = processor.START_IDX["length"] + processor.RANGE_LEN
    new_x = multiply_sequence(new_x, time_r_ints, len_lb, len_ub)
    new_y = multiply_sequence(new_y, time_r_ints, len_lb, len_ub)
    
    return new_x.to(device), new_y.to(device), {k: v.to(device) for k, v in metadata.items()}

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y, metadata = get_batch(split) 
            logits, loss = m(X, metadata, Y)  # Pass metadata to the model
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

# Print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    if iter % config['save_interval'] == config['save_interval'] - 1:
        torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')

    # Evaluate the loss on train and val sets at intervals
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb, metadata = get_batch('train')

    # Evaluate the loss
    logits, loss = m(xb, metadata, yb)  # Pass metadata to the model
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


6.579589 M parameters
step 0: train loss 6.1511, val loss 6.1539
step 100: train loss 3.6358, val loss 3.5867
step 200: train loss 3.5392, val loss 3.4879
step 300: train loss 3.4079, val loss 3.3741
step 400: train loss 3.2716, val loss 3.2327
step 500: train loss 3.2122, val loss 3.1779
step 600: train loss 3.1567, val loss 3.1079


In [ ]:
# torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')
# m = load_model('models/train_2.6325-val_3.1961.pth')

c:\Users\Draco\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
# Set the composer metadata
composers = ['Bach', 'Beethoven', 'Chopin', 'Liszt', 'Mozart', 'Scarlatti', 'Schubert']
composer_name = "Bach"  # Choose the composer you want
composer_index = composers.index(composer_name)

# Prepare metadata input with the specified composer
metadata = {
    "composer": torch.tensor([composer_index]).to(device)  # Metadata for a batch of 1
}
# generate from the model
context = torch.tensor([[32, 44, 56]], device=device)
with torch.no_grad():
    generated_sequence = m.generate(context, metadata, 200).tolist()

In [12]:
processor.decode_midi(generated_sequence[0], 'thing.mid')